In [1]:
from dr_manhattan.exchanges import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os

In [2]:
config = {
    "private_key": os.getenv("POLYMARKET_PRIVATE_KEY"),
    "funder": os.getenv("POLYMARKET_FUNDER"),
}

exchange = Polymarket(config)
print(f"Polymarket time-series ready ({'authenticated' if exchange.private_key else 'public'})")

Polymarket time-series ready (authenticated)


In [3]:
crypto_tag = exchange.get_tag_by_slug('crypto')
crypto_hourly = exchange.get_tag_by_slug('1H')
crypto_15min = exchange.get_tag_by_slug('15M')
crypto_daily = exchange.get_tag_by_slug('today')
crypto_hit = exchange.get_tag_by_slug('hit-price')
election = exchange.get_tag_by_slug('election')

crypto_15min

Tag(id='102467', label='15M', slug='15M', force_show=None, force_hide=None, is_carousel=None, published_at=None, created_at='2025-08-06T16:02:23.483913Z', updated_at='2025-09-13T03:05:05.692446Z', raw={'id': '102467', 'label': '15M', 'slug': '15M', 'createdAt': '2025-08-06T16:02:23.483913Z', 'updatedAt': '2025-09-13T03:05:05.692446Z'})

In [28]:
crypto_markets = exchange.search_markets(
    limit=1000,
    tag_id=crypto_hourly.id,
    closed=False,
    keywords = ['bitcoin', 'up or down'],
    log=True
)

crypto_markets

current-offset: 0
page_limit: 200
----------
current-offset: 200
page_limit: 200
----------


[Market(id='805591', question='Bitcoin Up or Down - December 5, 2AM ET', outcomes=['Up', 'Down'], close_time=datetime.datetime(2025, 12, 5, 8, 0, tzinfo=datetime.timezone.utc), volume=0.0, liquidity=0.0, prices={}, metadata={'id': '805591', 'question': 'Bitcoin Up or Down - December 5, 2AM ET', 'conditionId': '0x9ba80eae755ffca4d92b02a7eea0e5d4c602f5a23fa3f249c7f3b68dc6891d65', 'slug': 'bitcoin-up-or-down-december-5-2am-et', 'resolutionSource': 'https://www.binance.com/en/trade/BTC_USDT', 'endDate': '2025-12-05T08:00:00Z', 'liquidity': '0', 'startDate': '2025-12-03T07:01:07.25067Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png', 'description': 'This market will resolve to "Up" if the close price is greater than or equal to the open price for the BTC/USDT 1 hour candle that begins on the time and date specified in the title. Otherwise, this market will resolve to "Down".\n

In [23]:
epl_tag = exchange.get_tag_by_slug('EPL')

tottenham_markets = exchange.search_markets(
        limit=200,
        tag_id=epl_tag.id,
        closed=False,
        keywords=['will', 'win on', 'manchester united'],
        log=True
    )

tottenham_markets

current-offset: 0
page_limit: 200
----------


[Market(id='793577', question='Will Manchester United FC win on 2025-12-15?', outcomes=['Yes', 'No'], close_time=datetime.datetime(2025, 12, 15, 20, 0, tzinfo=datetime.timezone.utc), volume=1.234563, liquidity=166.9661, prices={'Yes': 0.45, 'No': 0.55}, metadata={'id': '793577', 'question': 'Will Manchester United FC win on 2025-12-15?', 'conditionId': '0x9c27e486e85b10f45a7775ab3e5b83efdd35a511866efadf74fbc1f2dbc9aee1', 'slug': 'epl-mun-bou-2025-12-15-mun', 'resolutionSource': 'https://www.premierleague.com/', 'endDate': '2025-12-15T20:00:00Z', 'liquidity': '166.9661', 'startDate': '2025-12-02T05:13:53.742326Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/Repetitive-markets/premier+league.jpg', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/Repetitive-markets/premier+league.jpg', 'description': 'In the upcoming game, scheduled for December 15, 2025\nIf Manchester United FC wins, this market will resolve to "Yes".\nOtherwise, this market will resolve t

In [ ]:
target_market = crypto_markets[0]
# target_market = tottenham_markets[0]

df = exchange.fetch_price_history(target_market, interval="max", fidelity=1, as_dataframe=True)

plt.plot(df['timestamp'], df['price'])
plt.xticks(rotation=45)
plt.show()

In [ ]:
trades = exchange.fetch_public_trades(market=target_market, limit=10000, as_dataframe=True, log=True)

trades.set_index('timestamp')
up_trades = trades[trades['outcome_index']==0].set_index('timestamp').sort_index()
down_trades = trades[trades['outcome_index']==1].set_index('timestamp').sort_index()
plt.plot(up_trades['price']['2025-11-29 19':])
plt.xticks(rotation=45)
plt.show()

In [ ]:
aespakarina = '0xca85f4b9e472b542e1df039594eeaebb6d466bf2'
#tmp_market='0xbb877d541d16896757c57107abe0632450ce1bedb4a46d4eae8ba1d1978207a6'
#user_trades = exchange.fetch_public_trades(market=tmp_market, user=aespakarina, limit=1000)
aespakarina_trades = exchange.fetch_public_trades(user=aespakarina, limit=1000, as_dataframe=True, log=True)
aespakarina_trades[:5]